In [1]:
import os
#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
from keras.datasets import mnist
from keras import models
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv1D, Conv2D, MaxPooling2D, GlobalMaxPooling1D, MaxPool1D, MaxPooling1D, GlobalMaxPool1D, SpatialDropout1D, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM
from keras.utils import to_categorical
import tensorflow as tf
from keras import backend as K


import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import spacy
# from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score

Using TensorFlow backend.


In [0]:
# importing data and embeddings
X_train = np.load('glove_x_train_padded.npy')
y_train = np.load('y_train.npy')

X_valid = np.load('glove_x_valid_padded.npy')
y_valid = np.load('y_valid.npy')

glove_embeddings = np.load('glove_embedding_100d.npy')

In [0]:
max_features = glove_embeddings.shape[0]
maxlen = 150
batch_size = 128
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 50
embed_size = 100 # for glove we are using 100d dataset
n_class = 12

#### Using L2 Regularization - This is not having a DROPOUT in your Model. This is different!

In [4]:
model = Sequential()

model.add(Embedding(max_features, embed_size, weights=[glove_embeddings], trainable=False, input_length=maxlen))

model.add(Dropout(0.2))
model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu',
                 strides=1))
model.add(MaxPooling1D())
model.add(Conv1D(filters, kernel_size, padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())

# L2 regularization
model.add(Dense(hidden_dims, activation = 'relu', kernel_regularizer=keras.regularizers.l2(0.001)))
model.add(Dense(hidden_dims, activation = 'relu', kernel_regularizer=keras.regularizers.l2(0.001)))
model.add(Dense(n_class, activation = 'sigmoid'))


model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          1191900   
_________________________________________________________________
dropout_1 (Dropout)          (None, 150, 100)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 148, 250)          75250     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 74, 250)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 72, 250)           187750    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 36, 250)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9000)             

In [5]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train Model
model.fit(X_train, y_train, batch_size=batch_size, epochs=50,class_weight='auto'  ,validation_split=0.15)

Train on 8819 samples, validate on 1557 samples
Epoch 1/50
8819/8819 [==============================] - 8s 939us/step - loss: 0.6095 - accuracy: 0.8801 - val_loss: 0.4123 - val_accuracy: 0.8945
Epoch 2/50
8819/8819 [==============================] - 1s 110us/step - loss: 0.3639 - accuracy: 0.8976 - val_loss: 0.3203 - val_accuracy: 0.9019
Epoch 3/50
8819/8819 [==============================] - 1s 108us/step - loss: 0.3061 - accuracy: 0.9034 - val_loss: 0.2808 - val_accuracy: 0.9107
Epoch 4/50
8819/8819 [==============================] - 1s 106us/step - loss: 0.2722 - accuracy: 0.9117 - val_loss: 0.2611 - val_accuracy: 0.9184
Epoch 5/50
8819/8819 [==============================] - 1s 109us/step - loss: 0.2504 - accuracy: 0.9189 - val_loss: 0.2488 - val_accuracy: 0.9208
Epoch 6/50
8819/8819 [==============================] - 1s 107us/step - loss: 0.2368 - accuracy: 0.9221 - val_loss: 0.2414 - val_accuracy: 0.9228
Epoch 7/50
8819/8819 [==============================] - 1s 105us/step - loss

## Validation

In [8]:
score = model.evaluate(X_valid,y_valid)
score

2594/2594 [==============================] - 0s 102us/step


[0.3199956518646011, 0.9249230027198792]

## Precision & Recall

In [9]:
pred = model.predict(X_valid, batch_size=batch_size, verbose=1)

predictions = pred
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_valid, pred, average='micro')
    recall = recall_score(y_valid, pred, average='micro')
    f1 = f1_score(y_valid, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
    print('\n')

2594/2594 [==============================] - 0s 65us/step
For threshold:  0.1
Micro-average quality numbers
Precision: 0.5223, Recall: 0.7532, F1-measure: 0.6169


For threshold:  0.2
Micro-average quality numbers
Precision: 0.5921, Recall: 0.7153, F1-measure: 0.6479


For threshold:  0.3
Micro-average quality numbers
Precision: 0.6293, Recall: 0.6866, F1-measure: 0.6567


For threshold:  0.4
Micro-average quality numbers
Precision: 0.6593, Recall: 0.6578, F1-measure: 0.6586


For threshold:  0.5
Micro-average quality numbers
Precision: 0.6910, Recall: 0.6404, F1-measure: 0.6648


For threshold:  0.6
Micro-average quality numbers
Precision: 0.7194, Recall: 0.6213, F1-measure: 0.6668


For threshold:  0.7
Micro-average quality numbers
Precision: 0.7483, Recall: 0.5948, F1-measure: 0.6628


For threshold:  0.8
Micro-average quality numbers
Precision: 0.7736, Recall: 0.5658, F1-measure: 0.6536


For threshold:  0.9
Micro-average quality numbers
Precision: 0.8111, Recall: 0.5174, F1-measur